In [1]:
from datetime import datetime, timedelta
import dateutil
import shutil
from typing import Iterable
from pprint import pprint
import time
import pathlib

import numpy as np
import pandas as pd
import cloudcatalog
import cdflib
import matplotlib.pyplot as plt

In [2]:
mms_id = 1
EARTH_RADIUS = 6378.14  # km

In [3]:
def progressbar(iterator: Iterable, iter_length: int = None, text: str = None):
    """
    A terminal progress bar.

    Parameters
    ----------
    iterator: Iterable
        The iterable that will be looped over.
    iter_length: int
        How many items the iterator loop over. If None, will calculate it
        using len(iterator).
    text: str
        Insert an optional text string in the beginning of the progressbar.
    """
    if text is None:
        text = ''
    else:
        text = text + ':'

    if iter_length is None:
        iter_length = len(iterator)

    try:
        for i, item in enumerate(iterator):
            i += 1  # So we end at 100%. Happy users!
            terminal_cols = shutil.get_terminal_size(fallback=(80, 20)).columns
            max_cols = int(terminal_cols - len(text) - 10)
            # Prevent a crash if the terminal window is narrower then len(text).
            if max_cols < 0:
                max_cols = 0

            percent = round(100 * i / iter_length)
            bar = "#" * int(max_cols * percent / 100)
            print(f'{text} |{bar:<{max_cols}}| {percent}%', end='\r')
            yield item
    finally:
        print()  # end with a newline.


In [4]:
# open up the global Catalog
cr = cloudcatalog.CatalogRegistry()
endpoint = cr.get_endpoint("GSFC HelioCloud Public Temp")
fr = cloudcatalog.CloudCatalog(endpoint, cache=True)

In [5]:
fgm_catalog_meta = [i for i in fr.catalog['catalog'] if f'mms{mms_id}_fgm_srvy' in i['id']]
assert len(fgm_catalog_meta) == 1, f'{len(fgm_catalog_meta)}. MMS{mms_id} FGM catalogs found.'
fgm_catalog_meta = fgm_catalog_meta[0]

In [6]:
fgm_catalog_meta

{'id': 'mms1_fgm_srvy',
 'index': 's3://helio-public/MMS/mms1/fgm/srvy/l2/',
 'title': 'mms1/fgm/srvy/l2/',
 'start': '2015-06-01T00:00:00Z',
 'stop': '2021-12-31T23:59:00Z',
 'modification': '2023-03-08T00:00:00Z',
 'indextype': 'csv',
 'filetype': 'cdf'}

## Figure out the start and end dates for our for loop

In [7]:
# -1 because we don't need the UTC timezone letter.
start_date = dateutil.parser.parse(fgm_catalog_meta['start'][:-1])

save_dir = pathlib.Path(pathlib.Path.home() / 'mms_cs', 'data')
ns_path = save_dir / 'mms_ns_crossings.csv'
ns_progress_path = save_dir / 'mms_ns_progress.txt'

if ns_path.exists() and ns_progress_path.exists():
    ns_df = pd.read_csv(ns_path) 
    start_date = dateutil.parser.parse(ns_progress_path.read_text()) + timedelta(days=1)
else:
    ns_df = pd.DataFrame()
    start_date = dateutil.parser.parse(fgm_catalog_meta['start'][:-1])

print(f'{start_date=}')
end_date = dateutil.parser.parse(fgm_catalog_meta['stop'][:-1])
    
dates = [start_date + timedelta(days=i) for i in range((end_date-start_date).days)]

start_date=datetime.datetime(2021, 12, 30, 0, 0)


## Neutral sheet crossings
Here we calculate neutral sheet crossings using the simple criteria of $B_{x}$ changing sign. Furthermore, we impose the conditions that $r_{x}$ < 0 and $r>6$, filtering to events to times when MMS was on the nightside and beyond the inner magnetosphere.

In [8]:
def calc_bx_crossings(fgm: cdflib.CDF, mec: cdflib.CDF):
    """
    Calculate the time of Bx crossings, as well as MMS's location at the time.
    
    Parameters
    ----------
    fgm
        The Level 2 FGM survey data
    mec
        The MEC data
    """
    mms_id = fgm.globalattsget()['Source_name'][-1]
    assert isinstance(fgm, cdflib.CDF)
    assert isinstance(mec, cdflib.CDF)
    data = {}
    changed_sign_idx = np.where(
                    np.sign(fgm[f'mms{mms_id}_fgm_b_gsm_srvy_l2'][:, 0])[1:] - \
                    np.sign(fgm[f'mms{mms_id}_fgm_b_gsm_srvy_l2'][:, 0])[:-1] != 0
                    )[0]
    if changed_sign_idx.shape[0] == 0:
        data['time'] = np.array([])
        return data
    
    data['time'] = np.array(cdflib.cdfepoch.to_datetime(fgm['epoch']))[changed_sign_idx]
    data['bx'] = fgm[f'mms{mms_id}_fgm_b_gsm_srvy_l2'][changed_sign_idx, 0]
    data['by'] = fgm[f'mms{mms_id}_fgm_b_gsm_srvy_l2'][changed_sign_idx, 1]
    data['bz'] = fgm[f'mms{mms_id}_fgm_b_gsm_srvy_l2'][changed_sign_idx, 2]
    data['b'] = np.sqrt(data['bx']**2 + data['by']**2 + data['bz']**2)

    mec_times = np.array(cdflib.cdfepoch.to_datetime(mec['epoch']))
    changed_sign_idx_mec = -999_999*np.ones_like(changed_sign_idx)
    for i, _time in enumerate(data['time']):
        changed_sign_idx_mec[i] = np.argmin(np.abs((_time - mec_times)/pd.Timedelta(seconds=1)))
    data['rx'] = mec[f'mms{mms_id}_mec_r_gsm'][changed_sign_idx_mec, 0]/EARTH_RADIUS
    data['ry'] = mec[f'mms{mms_id}_mec_r_gsm'][changed_sign_idx_mec, 1]/EARTH_RADIUS
    data['rz'] = mec[f'mms{mms_id}_mec_r_gsm'][changed_sign_idx_mec, 2]/EARTH_RADIUS
    data['r'] = np.sqrt(data['rx']**2 + data['ry']**2 + data['rz']**2)
    
    idx = np.where((data['r'] > 6) & (data['rx'] < 0))[0]
    for key, val in data.items():
        data[key] = val[idx]
    return data

In [9]:
skipped_dates = []
start_time = time.time()

for _start_date, _end_date in progressbar(zip(dates[:-1], dates[1:]), iter_length=len(dates)-1):
    _start_date = _start_date.isoformat()
    _end_date = _end_date.isoformat()
    fgm_cat = fr.request_cloud_catalog(f'mms{mms_id}_fgm_srvy', start_date=_start_date, stop_date=_end_date)
    mec_cat = fr.request_cloud_catalog(f'mms{mms_id}_mec_srvy_epht89d', start_date=_start_date, stop_date=_end_date)
    
    if len(fgm_cat['datakey']) == 0:  # No FGM files.
        skipped_dates.append(_start_date)
        continue
    fgm = cdflib.CDF(fgm_cat['datakey'].values[0])
    mec = cdflib.CDF(mec_cat['datakey'].values[0])
    data = calc_bx_crossings(fgm, mec)
    if data['time'].shape[0] == 0:
        ns_progress_path.write_text(_start_date)
        continue
    ns_df = pd.concat((ns_df, pd.DataFrame(data))).round(decimals=2)
    ns_df.to_csv(ns_path, index=False)
    ns_progress_path.write_text(_start_date)
    
print(f'Skipped dates: {len(skipped_dates)} | run time: {(time.time() - start_time)//3600} hours.')


Skipped dates: 0 | run time: 0.0 hours.
